In [144]:
import os
import cv2
import numpy as np
import csv
import glob as glob
from matplotlib import pyplot as plt
import xml.etree.ElementTree as ET
from lxml import etree

source = "MW-R"
OUTPUT_DIR ="output"
INPUT_DIR = "input"
color = (255, 0, 0) 
  
# Line thickness of 2 px 
thickness = 2


def write_xml(xml_str, xml_path):
    # remove blank text before prettifying the xml
    parser = etree.XMLParser(remove_blank_text=True)
    root = etree.fromstring(xml_str, parser)
    # prettify
    xml_str = etree.tostring(root, pretty_print=True)
    # save to file
    with open(xml_path, 'wb') as temp_xml:
        temp_xml.write(xml_str)


def create_PASCAL_VOC_xml(xml_path, abs_path, folder_name, image_name, img_height, img_width, depth):
    # By: Jatin Kumar Mandav
    annotation = ET.Element('annotation')
    ET.SubElement(annotation, 'folder').text = folder_name
    ET.SubElement(annotation, 'filename').text = image_name
    ET.SubElement(annotation, 'path').text = abs_path
    source = ET.SubElement(annotation, 'source')
    ET.SubElement(source, 'database').text = 'Unknown'
    size = ET.SubElement(annotation, 'size')
    ET.SubElement(size, 'width').text = img_width
    ET.SubElement(size, 'height').text = img_height
    ET.SubElement(size, 'depth').text = depth
    ET.SubElement(annotation, 'segmented').text = '0'
    print(xml_path)
    xml_str = ET.tostring(annotation)
    write_xml(xml_str, xml_path)

def voc_format(class_name, point_1, point_2):
    # Order: class_name xmin ymin xmax ymax
    xmin, ymin = min(point_1[0], point_2[0]), min(point_1[1], point_2[1])
    xmax, ymax = max(point_1[0], point_2[0]), max(point_1[1], point_2[1])
    items = map(str, [class_name, xmin, ymin, xmax, ymax])
    return items
    
def append_bb(ann_path, line, extension):
    if '.txt' in extension:
        with open(ann_path, 'a') as myfile:
            myfile.write(line + '\n') # append line
    if '.xml' in extension:

        class_name, xmin, ymin, xmax, ymax = line

        tree = ET.parse(ann_path)
        annotation = tree.getroot()

        obj = ET.SubElement(annotation, 'object')
        ET.SubElement(obj, 'name').text = class_name
        ET.SubElement(obj, 'pose').text = 'Unspecified'
        ET.SubElement(obj, 'truncated').text = '0'
        ET.SubElement(obj, 'difficult').text = '0'

        bbox = ET.SubElement(obj, 'bndbox')
        ET.SubElement(bbox, 'xmin').text = xmin
        ET.SubElement(bbox, 'ymin').text = ymin
        ET.SubElement(bbox, 'xmax').text = xmax
        ET.SubElement(bbox, 'ymax').text = ymax

        xml_str = ET.tostring(annotation)
        write_xml(xml_str, ann_path)


def get_annotation_paths(img_path, annotation_formats):
    annotation_paths = []
    for ann_dir, ann_ext in annotation_formats.items():
        new_path = os.path.join(OUTPUT_DIR, ann_dir)
        new_path = img_path.replace(INPUT_DIR, new_path, 1)
        pre_path, img_ext = os.path.splitext(new_path)
        new_path = new_path.replace(img_ext, ann_ext, 1)
        annotation_paths.append(new_path)
    return annotation_paths

def save_bounding_box(annotation_paths, class_index, point_1, point_2, width, height):
    for ann_path in annotation_paths:
        # if '.txt' in ann_path:
        #     line = yolo_format("person", point_1, point_2, width, height)
        #     append_bb(ann_path, line, '.txt')
        print(ann_path)
        if '.xml' in ann_path:
            line = voc_format("person", point_1, point_2)
            append_bb(ann_path, line, '.xml')

annotation_formats = {'PASCAL_VOC' : '.xml', 'YOLO_darknet' : '.txt'}
annotation_formats = {'PASCAL_VOC' : '.xml'}

vid_folder = './Meeting'

csvs = glob.glob(source+"/*.csv")


In [147]:
print(csvs)

['MW-R\\MW-18Mar-31.csv']


In [148]:
for filename in csvs:
    video_file =filename[:-4]+".avi"
    # Create a VideoCapture object
    cap = cv2.VideoCapture(video_file)
    # Check if camera opened successfully
    if (cap.isOpened() == False): 
        print("Unable to read camera feed")
    
    dataFrame =pd.read_csv(filename, sep=",", header=None)
    dataFrame.columns = ["frame_no","detection","x","y","w","h","xx","yy","ww","hh"]
    # Default resolutions of the frame are obtained.The default resolutions are system dependent.
    # We convert the resolutions from float to integer.
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    output_file=video_file[:-4]+"output.avi"
    # Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
    out = cv2.VideoWriter(output_file,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
    print(filename)
    file_path, file_extension = os.path.splitext(video_file)
    file_extension = file_extension.replace('.', '_')
    file_path += file_extension
    video_name_ext = os.path.basename(file_path)
    if not os.path.exists(file_path):
        print(' Converting video to individual frames...')
        os.makedirs(file_path)
    i=0
    while(True):
        ret, frame = cap.read()
        print(ret)
        print(i)
        if ret == True:
            frame_out = frame.copy()
            selectedbox=dataFrame[dataFrame["frame_no"]==i]
#             selectedbox= dataFrame[dataFrame[i] == i]
            print(selectedbox)
            frame_name =  '{}_{}{}'.format(video_name_ext, i, desired_img_format)
            frame_path = os.path.join(file_path, frame_name)
            print(frame_name+"helloooooooooooooo")
            cv2.imwrite(frame_path, frame)
            
            annotation_paths = get_annotation_paths(frame_path, annotation_formats)
            abs_path = os.path.abspath(frame_path)
            folder_name = os.path.dirname(frame_path)
            image_name = os.path.basename(frame_path)

            img_height, img_width, depth = (str(number) for number in frame.shape)
            for ann_path_ in annotation_paths:
                    if not os.path.isfile(ann_path_):
                        create_PASCAL_VOC_xml(ann_path_, abs_path, folder_name, image_name, img_height, img_width, depth)
            boxes = selectedbox.iloc[:,2:6]
            for index, row in boxes.iterrows():
                box = list(row)
                print(list(row))
                start_point = (int(box[0]),int(box[1]))
                end_point = (int(box[0])+int(box[2]),int(box[1])+int(box[3]))
                xx = start_point[0]
                yy = start_point[1]
                ww = end_point[0]
                hh = end_point[1]
                save_bounding_box(annotation_paths, 0, (xx, yy),(ww, hh), img_width, img_height)
                frame = cv2.rectangle(frame, start_point, end_point, color, thickness)
            out.write(frame)
            cv2.imshow('frame',frame)
            i = i + 1
        # Press Q on keyboard to stop recording
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Break the loop
        else:
            break  

    # When everything done, release the video capture and video write objects
    cap.release()
    out.release()

    # Closes all the frames
    cv2.destroyAllWindows() 

MW-R\MW-18Mar-31.csv
 Converting video to individual frames...
True
0
   frame_no  detection    x    y    w    h  xx  yy  ww  hh
0         0          1  482    0  120  200   1  -1  -1  -1
1         0          2  354   30   98  150   1  -1  -1  -1
2         0          3  512  872   72   86   1  -1  -1  -1
3         0          4  198  178  206  214   1  -1  -1  -1
MW-18Mar-31_avi_0.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_0.xml
[482, 0, 120, 200]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_0.xml
[354, 30, 98, 150]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_0.xml
[512, 872, 72, 86]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_0.xml
[198, 178, 206, 214]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_0.xml
True
1
   frame_no  detection    x    y    w    h  xx  yy  ww  hh
4         1          1  482    0  118  194   1  -1  -1  -1
5         1          2  356   28   96  146   1  -1  -1  -1
6         1          3  514  868   72   88   1  -1  -1  -1
7         1          4  200  170  202  212   1  -1  -1  -1


    frame_no  detection    x    y    w    h  xx  yy  ww  hh
52        13          1  500    0   92  132   1  -1  -1  -1
53        13          2  392    6   70  118   1  -1  -1  -1
54        13          3  540  842   78  112   1  -1  -1  -1
55        13          4  218  106  176  176   1  -1  -1  -1
MW-18Mar-31_avi_13.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_13.xml
[500, 0, 92, 132]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_13.xml
[392, 6, 70, 118]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_13.xml
[540, 842, 78, 112]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_13.xml
[218, 106, 176, 176]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_13.xml
True
14
    frame_no  detection    x    y    w    h  xx  yy  ww  hh
56        14          1  500    0   92  130   1  -1  -1  -1
57        14          2  394    4   70  118   1  -1  -1  -1
58        14          3  542  840   78  114   1  -1  -1  -1
59        14          4  218  104  174  174   1  -1  -1  -1
MW-18Mar-31_avi_14.jpghelloooooooooooooo
MW-R\MW-18Mar

     frame_no  detection    x    y    w    h  xx  yy  ww  hh
104        26          1  512    0   84  100   1  -1  -1  -1
105        26          2  422    0   62   88   1  -1  -1  -1
106        26          3  520  798   82  154   1  -1  -1  -1
107        26          4  212   96  150  148   1  -1  -1  -1
MW-18Mar-31_avi_26.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_26.xml
[512, 0, 84, 100]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_26.xml
[422, 0, 62, 88]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_26.xml
[520, 798, 82, 154]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_26.xml
[212, 96, 150, 148]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_26.xml
True
27
     frame_no  detection    x    y    w    h  xx  yy  ww  hh
108        27          1  512    0   84   98   1  -1  -1  -1
109        27          2  424    0   62   86   1  -1  -1  -1
110        27          3  516  796   82  154   1  -1  -1  -1
111        27          4  210   96  146  150   1  -1  -1  -1
MW-18Mar-31_avi_27.jpghelloooooooooooooo
MW-R\

[188, 108, 126, 144]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_38.xml
True
39
     frame_no  detection    x    y    w    h  xx  yy  ww  hh
156        39          1  524    0   72   74   1  -1  -1  -1
157        39          2  444    0   56   62   1  -1  -1  -1
158        39          3  392  782   98  156   1  -1  -1  -1
159        39          4  188  110  124  140   1  -1  -1  -1
MW-18Mar-31_avi_39.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_39.xml
[524, 0, 72, 74]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_39.xml
[444, 0, 56, 62]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_39.xml
[392, 782, 98, 156]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_39.xml
[188, 110, 124, 140]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_39.xml
True
40
     frame_no  detection    x    y    w    h  xx  yy  ww  hh
160        40          1  524    0   72   72   1  -1  -1  -1
161        40          2  446    0   54   60   1  -1  -1  -1
162        40          3  382  782  100  156   1  -1  -1  -1
163        40          4  186  110

MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_52.xml
[534, 0, 62, 56]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_52.xml
[458, 0, 50, 42]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_52.xml
[376, 814, 86, 142]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_52.xml
[176, 120, 124, 126]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_52.xml
True
53
     frame_no  detection    x    y    w    h  xx  yy  ww  hh
212        53          1  534    0   62   54   1  -1  -1  -1
213        53          2  460    0   48   42   1  -1  -1  -1
214        53          3  382  818   84  138   1  -1  -1  -1
215        53          4  176  120  124  126   1  -1  -1  -1
MW-18Mar-31_avi_53.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_53.xml
[534, 0, 62, 54]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_53.xml
[460, 0, 48, 42]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_53.xml
[382, 818, 84, 138]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_53.xml
[176, 120, 124, 126]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_53.xml
True
54
     frame_no  detection    x    y    w    h  xx

...

[176, 120, 124, 126]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_292.xml
True
293
      frame_no  detection    x    y    w    h  xx  yy  ww  hh
1172       293          1  548    0   52   38   1  -1  -1  -1
1173       293          2  480    0   40   22   1  -1  -1  -1
1174       293          3  508  780   92  158   1  -1  -1  -1
1175       293          4  176  120  124  126   1  -1  -1  -1
MW-18Mar-31_avi_293.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_293.xml
[548, 0, 52, 38]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_293.xml
[480, 0, 40, 22]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_293.xml
[508, 780, 92, 158]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_293.xml
[176, 120, 124, 126]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_293.xml
True
294
      frame_no  detection    x    y    w    h  xx  yy  ww  hh
1176       294          1  548    0   52   38   1  -1  -1  -1
1177       294          2  480    0   40   22   1  -1  -1  -1
1178       294          3  508  778   92  160   1  -1  -1  -1
1179       294  

[548, 0, 52, 38]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_307.xml
[478, 0, 40, 22]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_307.xml
[510, 758, 100, 178]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_307.xml
[176, 120, 124, 126]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_307.xml
True
308
      frame_no  detection    x    y    w    h  xx  yy  ww  hh
1232       308          1  548    0   52   38   1  -1  -1  -1
1233       308          2  478    0   40   22   1  -1  -1  -1
1234       308          3  510  756  100  180   1  -1  -1  -1
1235       308          4  176  120  124  126   1  -1  -1  -1
MW-18Mar-31_avi_308.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_308.xml
[548, 0, 52, 38]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_308.xml
[478, 0, 40, 22]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_308.xml
[510, 756, 100, 180]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_308.xml
[176, 120, 124, 126]
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_308.xml
True
309
      frame_no  detection    x    y    w    h  xx  yy  ww  hh
1236       

Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-31_avi_323.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_323.xml
True
324
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-31_avi_324.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_324.xml
True
325
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-31_avi_325.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_325.xml
True
326
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-31_avi_326.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_326.xml
True
327
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-31_avi_327.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_327.xml
True
328
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-3

Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-31_avi_373.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_373.xml
True
374
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-31_avi_374.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_374.xml
True
375
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-31_avi_375.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_375.xml
True
376
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-31_avi_376.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_376.xml
True
377
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-31_avi_377.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_377.xml
True
378
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-3

True
422
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-31_avi_422.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_422.xml
True
423
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-31_avi_423.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_423.xml
True
424
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-31_avi_424.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_424.xml
True
425
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-31_avi_425.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_425.xml
True
426
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
MW-18Mar-31_avi_426.jpghelloooooooooooooo
MW-R\MW-18Mar-31_avi\MW-18Mar-31_avi_426.xml
True
427
Empty DataFrame
Columns: [frame_no, detection, x, y, w, h, xx, yy, ww, hh]
Index: []
M

In [135]:
data = pd.read_csv("D:\\notebooks\\MW-R\\MW-18Mar-31.csv", sep=",", header=None)
data.columns = ["frame_no","detection","x","y","w","h","xx","yy","ww","hh"]
# for i in range(0,max(data[0]+1)):
#     print(i)

In [142]:
selectedbox=data[data["frame_no"]==1]

,frame_no,detection,x,y,w,h,xx,yy,ww,hh
4,1,1,482,0,118,194,1,-1,-1,-1
5,1,2,356,28,96,146,1,-1,-1,-1
6,1,3,514,868,72,88,1,-1,-1,-1
7,1,4,200,170,202,212,1,-1,-1,-1


In [106]:
# data

In [128]:
data[data[0]==0]

,0,1,2,3,4,5,6,7,8,9
0,0,1,482,0,120,200,1,-1,-1,-1
1,0,2,354,30,98,150,1,-1,-1,-1
2,0,3,512,872,72,86,1,-1,-1,-1
3,0,4,198,178,206,214,1,-1,-1,-1


In [131]:
data.loc[data[1] == 1]

,0,1,2,3,4,5,6,7,8,9
0,0,1,482,0,120,200,1,-1,-1,-1
4,1,1,482,0,118,194,1,-1,-1,-1
8,2,1,484,0,116,188,1,-1,-1,-1
12,3,1,486,0,112,182,1,-1,-1,-1
16,4,1,486,0,112,176,1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...
1268,317,1,544,0,54,40,1,-1,-1,-1
1272,318,1,542,0,56,42,1,-1,-1,-1
1276,319,1,540,0,56,44,1,-1,-1,-1
1280,320,1,538,0,58,46,1,-1,-1,-1


In [134]:
elderly = data[0.:] == 0

TypeError: '(0, slice(None, None, None))' is an invalid key

In [133]:
elderly

0        True
1        True
2        True
3        True
4       False
        ...  
1283    False
1284    False
1285    False
1286    False
1287    False
Name: 0, Length: 1288, dtype: bool

In [105]:
boxes = selectedbox.iloc[:,2:6]

In [56]:
type(boxes)

pandas.core.frame.DataFrame

In [57]:
boxes

,2,3,4,5
320,536,0,60,46
321,480,0,40,22
322,544,826,78,128
323,176,120,124,126


In [66]:
for index, row in boxes.iterrows():
    print(list(row))

[536, 0, 60, 46]
[480, 0, 40, 22]
[544, 826, 78, 128]
[176, 120, 124, 126]


In [62]:
# for box in boxes.:
#     print(box[0])

In [120]:
data.iloc[:, 0:2][,]

0         0
1         0
2         0
3         0
4         1
       ... 
1283    320
1284    321
1285    321
1286    321
1287    321
Name: 0, Length: 1288, dtype: int64

In [116]:
for i in range(0,max(data[0]+1)):
    print(i)
    
#     selectedbox=data[data[0][i] == i]
# # selectedbox
# # # box = selectedbox.iloc[:,2:6]
# # # =list(box.iloc[1])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [109]:
selectedbox

,0,1,2,3,4,5,6,7,8,9
320,80,1,536,0,60,46,1,-1,-1,-1
321,80,2,480,0,40,22,1,-1,-1,-1
322,80,3,544,826,78,128,1,-1,-1,-1
323,80,4,176,120,124,126,1,-1,-1,-1


,0,1,2,3,4,5,6,7,8,9
320,80,1,536,0,60,46,1,-1,-1,-1
321,80,2,480,0,40,22,1,-1,-1,-1
322,80,3,544,826,78,128,1,-1,-1,-1
323,80,4,176,120,124,126,1,-1,-1,-1


[480, 0, 40, 22]